# Vietnamese Legal LLaMA 3.2 3B - Inference with vLLM & Unsloth

Chạy inference với model đã fine-tune trên các câu hỏi pháp lý tiếng Việt sử dụng **vLLM** và **Unsloth** để tối ưu tốc độ và hiệu suất.

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>
</div>

**Model**: `mikeethanh/vietnamese-legal-llama3.2-3b-merged`

**Features:**
- ⚡ Fast inference với vLLM
- 📚 Chuyên biệt về pháp lý Việt Nam
- 💬 Hỗ trợ chat interactive
- 📊 Batch processing cho nhiều câu hỏi

## 1. Install Required Libraries

Cài đặt các thư viện cần thiết cho inference trên Google Colab:

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"  # Enable extra 30% context lengths

# Check if running on Colab
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm transformers torch
else:
    # Colab-specific installation
    !pip install --upgrade -qqq uv
    try:
        import numpy, PIL
        get_numpy = f"numpy=={numpy.__version__}"
        get_pil = f"pillow=={PIL.__version__}"
    except:
        get_numpy = "numpy"
        get_pil = "pillow"

    try:
        import subprocess
        is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except:
        is_t4 = False

    # Select appropriate vLLM version based on GPU
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")

    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
    !uv pip install transformers==4.56.2

print("✅ Installation completed!")

## 2. Load Pre-trained Model with Unsloth

Load model đã được fine-tune cho pháp lý tiếng Việt với vLLM fast inference:

In [2]:
from unsloth import FastLanguageModel
import torch

# Model configuration
model_name = "mikeethanh/vietnamese-legal-llama3.2-3b-merged"
max_seq_length = 4096  # Increase for longer legal documents
dtype = torch.float16  # Use float16 for better performance

print(f"🔄 Loading model: {model_name}")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,  # Set to True if memory is limited
    fast_inference = True,  # Enable vLLM fast inference
    gpu_memory_utilization = 0.85,  # Adjust based on available GPU memory
)

print("✅ Model loaded successfully!")
print(f"📊 Model info: {model_name}")
print(f"🔤 Tokenizer vocab size: {tokenizer.vocab_size}")
print(f"📏 Max sequence length: {max_seq_length}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-26 08:29:54 [__init__.py:244] Automatically detected platform cuda.
ERROR 11-26 08:29:57 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
🔄 Loading model: mikeethanh/vietnamese-legal-llama3.2-3b-merged
INFO 11-26 08:30:06 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 11-26 08:30:06 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-26 08:30:30 [config.py:1472] Using max model len 4096
WARNING 11-26 08:30:30 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 11-26 08:30:32 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 11-26 08:30:32 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='mikeethanh/vietnamese-legal-llama3.2-3b-merged', speculative_config=None, tokenizer='mikeethanh/vietnamese-legal-llama3.2-3b-merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespac

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

✅ Model loaded successfully!
📊 Model info: mikeethanh/vietnamese-legal-llama3.2-3b-merged
🔤 Tokenizer vocab size: 128000
📏 Max sequence length: 4096


## 3. Setup System Prompt for Legal Q&A

Thiết lập system prompt và chat template tối ưu cho các câu hỏi pháp lý tiếng Việt:

In [3]:
# Vietnamese Legal Assistant System Prompt
vietnamese_legal_system_prompt = """Bạn là một trợ lý pháp lý AI chuyên nghiệp, có kiến thức sâu về pháp luật Việt Nam. Hãy trả lời các câu hỏi pháp lý một cách chính xác, rõ ràng và hữu ích.

Nguyên tắc hoạt động:
1. Cung cấp thông tin pháp lý chính xác theo quy định của pháp luật Việt Nam
2. Trả lời bằng tiếng Việt rõ ràng, dễ hiểu
3. Phân tích vấn đề một cách logic và có cấu trúc
4. Đưa ra lời khuyên thực tế và khả thi
5. Khi cần thiết, đề xuất tham khảo thêm chuyên gia pháp lý

Lưu ý: Thông tin này chỉ mang tính chất tham khảo. Để có lời khuyên chính thức, hãy tham khảo luật sư hoặc cơ quan có thẩm quyền."""

def create_legal_prompt(question):
    """Create a formatted chat prompt for legal questions"""
    return [
        {"role": "system", "content": vietnamese_legal_system_prompt},
        {"role": "user", "content": question}
    ]

# Test the prompt creation
sample_question = "Quy trình ly hôn đơn phương trong pháp luật Việt Nam như thế nào?"
sample_messages = create_legal_prompt(sample_question)

print("✅ System prompt configured!")
print("\n📝 Sample prompt structure:")
for i, msg in enumerate(sample_messages):
    print(f"{i+1}. Role: {msg['role']}")
    print(f"   Content preview: {msg['content'][:100]}...")
    print()

✅ System prompt configured!

📝 Sample prompt structure:
1. Role: system
   Content preview: Bạn là một trợ lý pháp lý AI chuyên nghiệp, có kiến thức sâu về pháp luật Việt Nam. Hãy trả lời các ...

2. Role: user
   Content preview: Quy trình ly hôn đơn phương trong pháp luật Việt Nam như thế nào?...



## 4. Basic Inference with vLLM

Thực hiện inference cơ bản với vLLM SamplingParams:

In [ ]:
from vllm import SamplingParams
import time

def generate_legal_response(question, temperature=0.7, top_p=0.9, max_tokens=2048):
    """Generate response for a legal question using vLLM"""

    # Create messages
    messages = create_legal_prompt(question)

    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Configure sampling parameters
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        stop=["<|eot_id|>"]  # Stop at end of turn for Llama models
    )

    # Generate response
    start_time = time.time()
    outputs = model.fast_generate(
        [prompt],
        sampling_params=sampling_params,
        lora_request=None
    )
    generation_time = time.time() - start_time

    response = outputs[0].outputs[0].text

    return {
        "question": question,
        "response": response,
        "generation_time": generation_time,
        "tokens_generated": len(tokenizer.encode(response))
    }

def format_legal_response(response_text, line_length=80):
    """Format response text với xuống dòng phù hợp cho legal content"""
    import textwrap
    
    # Split by sentences first để tránh cắt giữa câu
    sentences = response_text.split('. ')
    formatted_sentences = []
    
    for i, sentence in enumerate(sentences):
        # Add period back (except for last sentence if it doesn't end with period)
        if i < len(sentences) - 1:
            sentence += '.'
        elif not sentence.endswith('.') and not sentence.endswith('!') and not sentence.endswith('?'):
            if sentence.strip():  # Only add period if sentence is not empty
                sentence += '.'
        
        # Wrap long sentences
        if len(sentence) > line_length:
            wrapped = textwrap.fill(sentence, width=line_length, 
                                  break_long_words=False, break_on_hyphens=False)
            formatted_sentences.append(wrapped)
        else:
            formatted_sentences.append(sentence)
    
    # Join sentences with appropriate spacing
    formatted_text = '\n'.join(formatted_sentences)
    
    # Clean up extra spaces and improve legal document formatting
    formatted_text = formatted_text.replace('\n\n\n', '\n\n')  # Remove triple newlines
    formatted_text = formatted_text.replace(':\n', ':\n')      # Keep colon formatting
    
    # Add extra line breaks after numbered items for better readability
    import re
    formatted_text = re.sub(r'(\d+\.\s)', r'\n\1', formatted_text)  # Add line break before numbers
    formatted_text = re.sub(r'([a-z]\)\s)', r'\n\1', formatted_text)  # Add line break before letters
    
    # Clean up any leading newlines
    formatted_text = formatted_text.strip()
    
    return formatted_text

# Test với câu hỏi mẫu
test_question = "Các loại phương tiện được giao thông trên đường bộ"

print("🔄 Generating response...")
result = generate_legal_response(test_question)

# Format response for better readability
formatted_response = format_legal_response(result['response'])

print("✅ Response generated!")
print("\n" + "="*80)
print(f"❓ Câu hỏi: {result['question']}")
print("\n" + "="*80)
print(f"💬 Trả lời:")
print(formatted_response)
print("\n" + "="*80)
print(f"⏱️  Thời gian: {result['generation_time']:.2f}s")
print(f"🔢 Tokens: {result['tokens_generated']}")
print(f"⚡ Tốc độ: {result['tokens_generated']/result['generation_time']:.1f} tokens/s")

🔄 Generating response...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

✅ Response generated!

❓ Câu hỏi: Các loại phương tiện được giao thông trên đường bộ

💬 Trả lời:
Căn cứ tại Điều 10 Luật Giao thông đường bộ 2008 quy định các loại phương tiện giao thông trên đường bộ như sau: - Xe ô tô; - Xe ô tô chở khách; - Xe ô tô chở hàng; - Xe ô tô chuyên dùng; - Xe ô tô chuyên dùng khác; - Xe ô tô chuyên dùng của lực lượng Công an nhân dân; - Xe ô tô chuyên dùng của lực lượng Quân đội nhân dân; - Xe ô tô chuyên dùng của cơ quan, tổ chức, đơn vị hành chính - tư pháp; - Xe ô tô chuyên dùng của tổ chức, cá nhân được cơ quan, tổ chức có thẩm quyền cho phép; - Xe mô tô; - Máy kéo; - Xe đạp; - Xe thô sơ; - Rơ moóc; - Phương tiện khác tham gia giao thông đường bộ.

⏱️  Thời gian: 5.78s
🔢 Tokens: 189
⚡ Tốc độ: 32.7 tokens/s


## 5. Enhanced Response Formatting

Test với nhiều loại câu hỏi pháp lý và format output dễ đọc hơn:

In [ ]:
# Test với nhiều câu hỏi pháp lý khác nhau
test_questions = [
    "Mức phạt cho việc vượt đèn đỏ đối với xe ô tô là bao nhiêu?",
    "Thủ tục đăng ký kết hôn cần những giấy tờ gì?",
    "Điều kiện để được cấp bằng lái xe hạng B1 là gì?"
]

def test_multiple_questions(questions):
    """Test multiple legal questions với formatted output"""
    results = []
    
    for i, question in enumerate(questions, 1):
        print(f"\n{'='*20} QUESTION {i}/{len(questions)} {'='*20}")
        print(f"❓ {question}")
        print("\n🔄 Generating response...")
        
        # Generate response
        result = generate_legal_response(question, temperature=0.3)  # Lower temp for more consistent legal answers
        formatted_response = format_legal_response(result['response'])
        
        print("💬 Trả lời:")
        print("-" * 60)
        print(formatted_response)
        print("-" * 60)
        print(f"⏱️  {result['generation_time']:.2f}s | 🔢 {result['tokens_generated']} tokens | ⚡ {result['tokens_generated']/result['generation_time']:.1f} tokens/s")
        
        results.append({
            "question": question,
            "formatted_response": formatted_response,
            "stats": result
        })
    
    return results

# Run tests
print("🚀 Testing multiple legal questions với enhanced formatting...")
test_results = test_multiple_questions(test_questions)

print(f"\n✅ Completed testing {len(test_results)} questions!")
print("📊 Summary:")
avg_time = sum(r['stats']['generation_time'] for r in test_results) / len(test_results)
avg_tokens = sum(r['stats']['tokens_generated'] for r in test_results) / len(test_results)
avg_speed = sum(r['stats']['tokens_generated']/r['stats']['generation_time'] for r in test_results) / len(test_results)

print(f"   Average time: {avg_time:.2f}s")
print(f"   Average tokens: {avg_tokens:.0f}")
print(f"   Average speed: {avg_speed:.1f} tokens/s")

## 6. Interactive Legal Q&A

Tạo function interactive để người dùng có thể hỏi trực tiếp:

In [ ]:
def interactive_legal_qa():
    """Interactive legal Q&A với formatted output"""
    print("🏛️  Vietnamese Legal AI Assistant")
    print("="*50)
    print("💡 Gõ câu hỏi pháp lý và nhận câu trả lời định dạng đẹp")
    print("💡 Gõ 'quit' hoặc 'exit' để thoát")
    print("="*50)
    
    conversation_history = []
    
    while True:
        try:
            # Get user input
            question = input("\n❓ Câu hỏi của bạn: ").strip()
            
            # Check for exit commands
            if question.lower() in ['quit', 'exit', 'thoát', 'q']:
                print("\n👋 Cảm ơn bạn đã sử dụng Vietnamese Legal AI Assistant!")
                break
            
            if not question:
                print("⚠️  Vui lòng nhập câu hỏi!")
                continue
                
            print(f"\n🔄 Đang xử lý câu hỏi: '{question}'...")
            
            # Generate response
            result = generate_legal_response(question, temperature=0.3)
            formatted_response = format_legal_response(result['response'])
            
            # Display formatted result
            print("\n" + "="*60)
            print("💬 VIETNAMESE LEGAL AI RESPONSE:")
            print("="*60)
            print(formatted_response)
            print("="*60)
            print(f"⏱️  Thời gian: {result['generation_time']:.2f}s | "
                  f"🔢 {result['tokens_generated']} tokens | "
                  f"⚡ {result['tokens_generated']/result['generation_time']:.1f} tokens/s")
            
            # Save to conversation history
            conversation_history.append({
                "question": question,
                "response": formatted_response,
                "timestamp": time.time(),
                "stats": result
            })
            
        except KeyboardInterrupt:
            print("\n\n👋 Đã dừng phiên chat. Cảm ơn bạn!")
            break
        except Exception as e:
            print(f"\n❌ Lỗi: {e}")
            print("🔄 Vui lòng thử lại...")
    
    # Show conversation summary
    if conversation_history:
        print(f"\n📊 Tóm tắt phiên chat:")
        print(f"   - Tổng số câu hỏi: {len(conversation_history)}")
        avg_time = sum(h['stats']['generation_time'] for h in conversation_history) / len(conversation_history)
        print(f"   - Thời gian trung bình: {avg_time:.2f}s/câu hỏi")
    
    return conversation_history

# Uncomment dòng dưới để bắt đầu interactive chat
# interactive_legal_qa()

print("✅ Interactive legal Q&A function ready!")
print("💡 Uncomment dòng 'interactive_legal_qa()' ở trên để bắt đầu chat")